# Using structured JSON output feature from OpenAI
Sources: 
- OpenAI web page for JSON formatting rules to use 'strict' mode: https://platform.openai.com/docs/guides/structured-outputs
- A Python Package for high-level usage of LLM structured output: https://github.com/dottxt-ai/outlines

In [30]:
# !pip uninstall -y sympy pysb torch outlines
# !pip install pysb sympy==1.11.1
# !pip install torch sympy==1.13.1 --no-deps
# !pip install outlines --no-deps

In [1]:
import sys
from pathlib import Path
import os
# get current path
sys.path.append(str(Path.cwd().parent))

## Providing the JSON schema directly as the response format
- I.e. Provide the indra JSON schema to the `response_format` parameter of `OpenAI.beta.chat.completions.parse` function of OpenAI

OpenAI enforces a subset of the 'JSON Schema' language. The below is a list of non-exhaustive rules that is enforced:
1. root JSON object is 'object' type
2. 'required' includes all and only the fields in 'property' 
    - This ensures LLM doesn't generate any key-val pair in 'property' that is not defined in the schema.
    - To make a defined field in 'property' optional, we can add "null" as an optional type of that field. 
3. 'additionalProperties' field is set to false and is included for every 'object' type.
4. Each sub-schema in 'anyOf', namely "#/definitions/RegulateActivity", "#/definitions/Modification", "#/definitions/SelfModification", etc. has a different first key
   in their 'property' field. <br>
   This is achievable by adding a new field with a unique constant value. E.g. "#/definitions/RegulateActivity" has 
   ```
   "kind": {
                "type": "string",
                "const": "RegulateActivity"
            }
    ```
    And "#/definitions/Modification" has
    ```
    "kind": {
                "type": "string",
                "const": "Modification"
            }
    ```

- The below schema follows all the rules for OpenAI 'strict' mode, except for one. It contains the 'allOf' syntax, which OpenAI doesn't currently support.
- We can resolve 'allOf' syntax by merging the items and removing the 'allOf' syntax.

In [2]:
import json
import copy

In [4]:
# Load the JSON schema
schema_path = "/Users/thomaslim/gyorilab/indra_gpt/indra_gpt/resources/indra_schema_openai.json"
with open(schema_path, "r") as f:
    schema = json.load(f)

In [5]:
def resolve_ref(schema, ref_path):
    """Helper function to resolve a $ref path in a JSON schema."""
    keys = ref_path.lstrip("#/").split("/")
    ref_obj = schema
    for key in keys:
        ref_obj = ref_obj.get(key, {})
    return copy.deepcopy(ref_obj)  # Return a deep copy to prevent modifying the original schema

def merge_allOf(schema, root_schema):
    """Recursively merges allOf definitions into their parent objects and removes allOf.
       - Resolves $ref only if inside allOf
       - Resolves nested allOf in properties, items, and definitions
       - Does NOT resolve nested $ref inside resolved object
    """
    if isinstance(schema, dict):
        if "allOf" in schema:
            merged_schema = {}
            required_fields = set()

            for sub_schema in schema["allOf"]:
                if "$ref" in sub_schema:
                    ref_obj = resolve_ref(root_schema, sub_schema["$ref"])
                    sub_schema = ref_obj.copy()  # Use a copy to prevent modifying the root schema

                # Merge properties correctly
                for key, value in sub_schema.items():
                    if key == "required":
                        required_fields.update(value)
                    elif key in merged_schema and isinstance(merged_schema[key], dict) and isinstance(value, dict):
                        merged_schema[key].update(value)  # Merge nested dictionaries (e.g., properties)
                    else:
                        merged_schema[key] = value  # Overwrite other keys

            merged_schema.pop("allOf", None)  # Remove allOf after merging
            if required_fields:
                merged_schema["required"] = list(required_fields)  # Assign merged required fields
            
            schema.clear()
            schema.update(merged_schema)

        # Recursively process properties, items, and definitions **after merging**
        for key in ["properties", "items", "definitions"]:
            if key in schema and isinstance(schema[key], dict):
                schema[key] = {k: merge_allOf(v, root_schema) for k, v in schema[key].items()}

        return schema

    elif isinstance(schema, list):
        return [merge_allOf(item, root_schema) for item in schema]

    return schema  # Return primitive values unchanged

In [6]:
# Resolve `allOf` occurrences while resolving only first-level refs inside 'allOf'
post_processed_schema = merge_allOf(schema, schema)

In [7]:
post_processed_schema

{'type': 'object',
 'properties': {'statements': {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/RegulateActivity'},
     {'$ref': '#/definitions/Modification'},
     {'$ref': '#/definitions/SelfModification'},
     {'$ref': '#/definitions/ActiveForm'},
     {'$ref': '#/definitions/Gef'},
     {'$ref': '#/definitions/Gap'},
     {'$ref': '#/definitions/Complex'},
     {'$ref': '#/definitions/Association'},
     {'$ref': '#/definitions/Translocation'},
     {'$ref': '#/definitions/RegulateAmount'},
     {'$ref': '#/definitions/Conversion'}]}}},
 'required': ['statements'],
 'additionalProperties': False,
 'definitions': {'RegulateActivity': {'type': 'object',
   'properties': {'kind': {'type': 'string', 'const': 'RegulateActivity'},
    'type': {'type': 'string',
     'enum': ['Activation', 'Inhibition'],
     'description': 'The type of the statement'},
    'subj': {'anyOf': [{'$ref': '#/definitions/Agent'}, {'type': 'null'}],
     'description': "The agent responsible

### Use `outlines` package to use OpenAI's 'structured outputs' feature.

In [9]:
import outlines

# Use OpenAI API instead of local model
model = outlines.models.openai("gpt-4o")  # Or "gpt-4-turbo" for cheaper API cost
# Define generators
generator_v1 = outlines.generate.json(model, json.dumps(post_processed_schema))
# Sample Prompt
prompt = "We found that 24 hours of BMP pretreatment caused a doubling in PTEN half-life (15.1 hours to 28.4 hours, p = 0.03, n = 4)."

In [10]:
result_v1 = generator_v1(prompt)

In [11]:
result_v1

{'statements': [{'kind': 'RegulateActivity',
   'type': 'Activation',
   'subj': {'name': 'BMP',
    'mods': None,
    'mutations': None,
    'bound_conditions': None,
    'activity': None,
    'location': None,
    'db_refs': None,
    'sbo': None},
   'obj': {'name': 'PTEN',
    'mods': None,
    'mutations': None,
    'bound_conditions': None,
    'activity': None,
    'location': None,
    'db_refs': None,
    'sbo': None},
   'obj_activity': 'stability',
   'evidence': [{'text': 'We found that 24 hours of BMP pretreatment caused a doubling in PTEN half-life (15.1 hours to 28.4 hours, p = 0.03, n = 4).',
     'source_api': None,
     'pmid': None,
     'source_id': None,
     'annotations': None,
     'epistemics': None}],
   'id': None,
   'supports': None,
   'supported_by': None}]}

Let us compare the above result with the actual JSON object from the benchmark dataset

In [12]:
indra_benchmark_corpus_sample_50 = json.load(open("/Users/thomaslim/gyorilab/indra_gpt/indra_gpt/resources/indra_benchmark_corpus_sample_50.json", "r"))
print(json.dumps(indra_benchmark_corpus_sample_50[0], indent=4))

{
    "type": "Activation",
    "subj": {
        "name": "BMP",
        "db_refs": {
            "FPLX": "BMP",
            "TEXT": "BMP"
        }
    },
    "obj": {
        "name": "PTEN",
        "db_refs": {
            "UP": "P60484",
            "HGNC": "9588",
            "TEXT": "PTEN"
        }
    },
    "obj_activity": "activity",
    "belief": 0.9145194475081968,
    "evidence": [
        {
            "source_api": "reach",
            "pmid": "21456062",
            "text": "We found that 24 hours of BMP pretreatment caused a doubling in PTEN half-life (15.1 hours to 28.4 hours, p = 0.03, n = 4).",
            "annotations": {
                "found_by": "Positive_activation_syntax_1_verb",
                "agents": {
                    "coords": [
                        [
                            26,
                            29
                        ],
                        [
                            64,
                            68
                   

In [13]:
from indra.statements.io import stmt_from_json

stmt_from_json(indra_benchmark_corpus_sample_50[0])

Activation(BMP(), PTEN())

In [51]:
# get the first 50 statements from the benchmark corpus
import pandas as pd
df = pd.DataFrame(indra_benchmark_corpus_sample_50)

In [ ]:
df = pd.read_csv("/Users/thomaslim/gyorilab/indra_gpt/indra_gpt/resources/positive_examples.tsv", sep="\t")

In [55]:
# Use generator_v1 and apply to text column, and add a new column called extracted_statement_json
df['extracted_statement_json'] = df['text'].apply(lambda x: generator_v1(x)['statements'])

INFO: [2025-02-07 10:30:35] openai._base_client - Retrying request to /chat/completions in 0.498217 seconds
INFO: [2025-02-07 10:30:39] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-07 10:30:39] openai._base_client - Retrying request to /chat/completions in 0.403070 seconds
INFO: [2025-02-07 10:30:48] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-07 10:30:48] openai._base_client - Retrying request to /chat/completions in 0.478104 seconds
INFO: [2025-02-07 10:31:05] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-07 10:31:05] openai._base_client - Retrying request to /chat/completions in 0.469419 seconds
INFO: [2025-02-07 10:31:18] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-07 10:31:18] openai._base_client - Retrying request to /chat/completions in 0.380128 seco

In [65]:
df.extracted_statement_json[0][0]

{'kind': 'SelfModification',
 'type': 'Autophosphorylation',
 'enz': {'name': 'PAK4',
  'mods': [{'mod_type': 'phosphorylation',
    'residue': 'Unknown',
    'position': 'Unknown',
    'is_modified': True}],
  'mutations': [],
  'bound_conditions': [],
  'activity': {'activity_type': 'kinase', 'is_active': True},
  'location': '',
  'db_refs': {},
  'sbo': 'SBO:0000176'},
 'residue': '',
 'position': '',
 'evidence': [{'text': 'PAK4 autophosphorylation observed at zero time point and subsequent time intervals.',
   'source_api': 'Experimental Data',
   'pmid': 'Pending',
   'source_id': 'LabNotes123',
   'annotations': {},
   'epistemics': {}}],
 'id': 'SelfMod1',
 'supports': [],
 'supported_by': []}

In [66]:
stmt_from_json(df.extracted_statement_json[0][0])

IndexError: list index out of range

In [57]:
# Now ally the indra.statements.io.stmt_from_json to convert the extracted_statement_json to a list of INDRA statements
df['extracted_statements'] = df['extracted_statement_json'].apply(lambda x: [stmt_from_json(y) for y in x])

IndexError: list index out of range